In [20]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse

# Thiết lập khóa API cho LlamaParse
api_key = 'llx-JUKgXwAJ4IrHHB1IFyDKh34IwHwPHiVPfyPbkXFIKTfkVhdS'

# Khởi tạo LlamaParse với chế độ 'continuous' và định dạng kết quả là 'xlsx'
parser = LlamaParse(api_key=api_key, result_type='markdown')

# Phân tích tệp .xlsx
documents = parser.load_data('/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx')

# Lưu kết quả ra tệp Excel mới
# with open('output.xlsx', 'wb') as f:
#     f.write(documents[0].get_content())
import pandas as pd

# Trích xuất dữ liệu structured từ documents
data = documents[0].get_content()

# Kiểm tra dữ liệu trước khi đưa vào DataFrame
print(data)

Started parsing the file under job_id 703fbb4e-1907-4632-9e97-64842ae80be9
# 1325

|TỔNG HỢP CÔNG NỢ OEM TUẦN| | | | | | | | | | | | | | | |8/03/2025| | | |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Tuần 1 tháng 3- từ ngày 01/3 đến 7/3/2025| | | | | | | | | | | | | | | | | | | |
|TT|Tên đại lý|Nợ ĐK|Có ĐK|PS Nợ|Thu tiền|Trả lại, CK|Nợ CK|Có CK|KẾ HOẠCH|Đặc điểm|Ngày PS|Đặt hàng|Tồn kho|Quá hạn tt|Ghi chú|Ghi chú|Ngày đưa xác nhận CN|Ngày nhận xác nhận CN|HĐNT|
|NỘI BỘ|HĐNT| | | | | | | | | | | | | | | | | | |
|THUẾ|CAM KẾT 2024| | | | | | | | | | | | | | | | | | |
|1|Nguyễn Thị Lan Thu|600,000,000|170,000,000|4,000,000,000|341,850,000|0|3,580,000,000|-8,150,000|0| | | | | | | | | | |
|1|OEMDUCTOAN| | | | | | | | | | | | | | | | | | |
|Phương: Đức Toàn| | | | | | | | | | | | | | | | | | | |
|Gold mark|100,000,000| | |41,850,000| | |-58,150,000| |Bản in 5tr| | | | | | | | | | |
|Đặt 30%| | | | | | | | | | | | | | | | | | | |
|TT : 1 tháng| | | | |

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama

# Khởi tạo Ollama LLM
llm = Ollama(model="llama2")

In [26]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=llm, num_workers=4)

In [27]:
nodes = node_parser.get_nodes_from_documents(documents)


1it [00:00, 5497.12it/s]


In [28]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [29]:
len(nodes), len(base_nodes), len(objects)


(3, 1, 1)

In [31]:
print(objects[0].get_content())

This table appears to be a list of OEM transactions for various companies, with information such as date, amount, and company name. The table has 10 rows and 15 columns, with each row representing a separate transaction.,
with the following table title:
OEM Transactions,
with the following columns:
- TỔNG HỢP CÔNG NỢ OEM TUẦN: None
- ---: None



In [35]:
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(model_name="llama2")  # hoặc mô hình khác mà bạn thích

recursive_index = VectorStoreIndex(
    nodes=base_nodes + objects, 
    llm=llm,
    embed_model=embed_model
)

recursive_query_engine_o1 = recursive_index.as_query_engine(
    similarity_top_k=5, 
    llm=llm
)


In [43]:
query = "Nam Thắng tổng công nợ bao nhiêu?"

response_recursive_o1 = recursive_query_engine_o1.query(query)

In [44]:
from IPython.display import Image, Markdown
display(Markdown(f"{response_recursive_o1}"))

Based on the provided table information, we can see that Nam Thắng's total debt is 500,000,000 VNĐ.

In [2]:
import os
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from llama_index.core import StorageContext, load_index_from_storage
from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.embeddings.ollama import OllamaEmbedding

class DataParser:
    def __init__(self, api_key, file_paths):
        self.api_key = api_key
        self.file_paths = file_paths
        self.parser = LlamaParse(api_key=self.api_key, result_type='markdown')

    def parse(self):
        documents = []
        for file_path in self.file_paths:
            documents.extend(self.parser.load_data(file_path))
        return documents

class NodeProcessor:
    def __init__(self, llm_model='llama2', num_workers=4):
        self.llm = Ollama(model=llm_model)
        self.node_parser = MarkdownElementNodeParser(llm=self.llm, num_workers=num_workers)

    def process_nodes(self, documents):
        base_nodes, objects = self.node_parser.get_nodes_and_objects(documents)
        return base_nodes, objects
    
class IndexBuilder:
    def __init__(self, index_path='storage',llm_model='llama2', embed_model_name='llama2'):
        self.llm = Ollama(model=llm_model)
        self.embed_model = OllamaEmbedding(model_name=embed_model_name)

    def build_index(self, nodes):
        return VectorStoreIndex(nodes=nodes, llm=self.llm, embed_model=self.embed_model)
    
    def build_or_load_index(self, nodes):
        if os.path.exists(self.index_path):
            storage_context = StorageContext.from_defaults(persist_dir=self.index_path)
            index = load_index_from_storage(storage_context)
        else:
            index = VectorStoreIndex(nodes=nodes, llm=self.llm, embed_model=self.embed_model)
            index.storage_context.persist(persist_dir=self.index_path)
        return index

class QueryEngine:
    def __init__(self, index, similarity_top_k=5, llm = None):
        self.query_engine = index.as_query_engine(similarity_top_k=similarity_top_k, llm = llm)

    def query(self, question):
        return self.query_engine.query(question)

def main():
    # Danh sách các file Excel cần xử lý
    excel_files = [
        '/home/namnh1/rag-llm-chatbot/BC AI/BC Nhân sự.xlsx',
        '/home/namnh1/rag-llm-chatbot/BC AI/BC Tiến độ sản xuất.xlsx',
        '/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx'
    ]
    
    # Khởi tạo và parse dữ liệu
    parser = DataParser(
        api_key='llx-JUKgXwAJ4IrHHB1IFyDKh34IwHwPHiVPfyPbkXFIKTfkVhdS',
        file_paths=excel_files
    )
    documents = parser.parse()    

    # Xử lý nodes
    processor = NodeProcessor()
    base_nodes, objects = processor.process_nodes(documents)

    # Xây dựng chỉ mục
    index_builder = IndexBuilder(index_path='storage')
    index = index_builder.build_index(base_nodes + objects)

    # Truy vấn
    query_engine = QueryEngine(index, similarity_top_k=5, llm =  index_builder.llm)
    response = query_engine.query("Công ty cổ phần S- Decoro có Giá trị HĐ bao nhiêu?")

    # Hiển thị kết quả
    print(f"{response}")
    
if __name__ == "__main__":
    main()

Started parsing the file under job_id 265fae7e-16f7-459f-801d-7fada5625b4a
Started parsing the file under job_id 2d35164e-7b9b-4bda-abda-bd68f906b85f
Started parsing the file under job_id acfd4a2e-4959-424d-9569-15e45d25348c
Thank you for providing more context. Based on the new information provided, we can estimate the total assets of S-Decoro as follows:

Total Assets = Total Resources - Total Liabilities

Total Resources = $50,000,000 (based on the transactions and accounts provided)
Total Liabilities = $25,000,000 (based on the transactions and accounts provided)

Therefore, the total assets of S-Decoro can be estimated as:

$50,000,000 - $25,000,000 = $25,000,000

So, the estimated total assets of S-Decoro are $25,000,000.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores import QdrantVectorStore  
from qdrant_client import QdrantClient                    

class DataParser:
    def __init__(self, api_key, file_path):
        self.api_key = api_key
        self.file_path = file_path
        self.parser = LlamaParse(api_key=self.api_key, result_type='markdown')

    def parse(self):
        return self.parser.load_data(self.file_path)

class NodeProcessor:
    def __init__(self, llm_model='gemma3:4b', num_workers=4):
        self.llm = Ollama(model=llm_model)
        self.node_parser = MarkdownElementNodeParser(llm=self.llm, num_workers=num_workers)

    def process_nodes(self, documents):
        base_nodes, objects = self.node_parser.get_nodes_and_objects(documents)
        return base_nodes, objects

class IndexBuilder:
    def __init__(self, 
                 llm_model='gemma3:4b', 
                 embed_model_name='gemma3:4b',
                 qdrant_host='localhost',
                 qdrant_port=6333,
                 collection_name='my_collection'):
        # Khởi tạo LLM + Embedding
        self.llm = Ollama(model=llm_model)
        self.embed_model = OllamaEmbedding(model_name=embed_model_name)
        
        # Khởi tạo Qdrant client
        self.qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
        
        # Tạo QdrantVectorStore
        self.vector_store = QdrantVectorStore(
            client=self.qdrant_client,
            collection_name=collection_name,
            embeddings=self.embed_model
        )

    def build_index(self, nodes):
        # Tạo index với vector_store là QdrantVectorStore
        return VectorStoreIndex(
            nodes=nodes, 
            llm=self.llm, 
            embed_model=self.embed_model, 
            vector_store=self.vector_store
        )

class QueryEngine:
    def __init__(self, index, similarity_top_k=5, llm=None):
        self.query_engine = index.as_query_engine(
            similarity_top_k=similarity_top_k, 
            llm=llm
        )

    def query(self, question):
        return self.query_engine.query(question)

def main():
    # Danh sách các file Excel cần xử lý
    excel_files = [
        '/home/namnh1/rag-llm-chatbot/BC AI/BC Nhân sự.xlsx',
        '/home/namnh1/rag-llm-chatbot/BC AI/BC Tiến độ sản xuất.xlsx',
        '/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx'
    ]
    
    # Khởi tạo và parse dữ liệu
    parser = DataParser(
        api_key='llx-JUKgXwAJ4IrHHB1IFyDKh34IwHwPHiVPfyPbkXFIKTfkVhdS', 
        file_path=excel_files
    )
    documents = parser.parse()

    # Xử lý nodes
    processor = NodeProcessor()
    base_nodes, objects = processor.process_nodes(documents)

    # Xây dựng chỉ mục (lưu vào Qdrant)
    index_builder = IndexBuilder(
        llm_model='llama2', 
        embed_model_name='llama2',
        qdrant_host='localhost',   # thay bằng host Qdrant của bạn
        qdrant_port=6333,          # thay bằng port Qdrant của bạn
        collection_name='my_collection'  # tùy chọn đặt tên collection
    )
    index = index_builder.build_index(base_nodes + objects)

    # Truy vấn
    query_engine = QueryEngine(
        index, 
        similarity_top_k=5, 
        llm=index_builder.llm
    )
    response = query_engine.query("Nam Thắng tổng công nợ bao nhiêu?")

    # Hiển thị kết quả
    from IPython.display import Markdown, display
    print(f"{response}")

if __name__ == "__main__":
    main()


In [2]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core import VectorStoreIndex
from llama_index.readers.file import PandasExcelReader
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# (Tuỳ chọn, nếu bạn vẫn muốn lưu vào Qdrant)
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

class DataParser:
    """
    Thay thế cho LlamaParse: Lớp này dùng PandasExcelReader từ llama_index
    để chuyển các file Excel thành list Document.
    """
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.reader = PandasExcelReader()

    def parse(self):
        documents = []
        for path in self.file_paths:
            docs = self.reader.load_data(file=path)
            documents.extend(docs)
        return documents
    
excel_files = [
    '/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx'
]

# 1. Tải dữ liệu Excel thành list Document
parser = DataParser(file_paths=excel_files)
documents = parser.parse()

In [8]:
documents[0].text_resource.text

'Tuần 1 tháng 3- từ ngày 01/3 đến 7/3/2025                     \nTT Tên đại lý Nợ ĐK Có ĐK PS Nợ Thu tiền Trả lại, CK Nợ CK Có CK KẾ HOẠCH Đặc điểm Ngày PS Đặt hàng Tồn kho Quá hạn tt Ghi chú Ghi chú Ngày đưa xác nhận CN Ngày nhận xác nhận CN HĐNT\nNỘI BỘ HĐNT\nTHUẾ CAM KẾT 2024\n1 Nguyễn Thị Lan Thu 600000000 170000000 4000000000 341850000 0 3580000000 -8150000 0            \n1 OEMDUCTOAN\nPhương: Đức Toàn\nGold mark 100000000   41850000   -58150000  Bản in 5tr\nĐặt 30%\nTT : 1 tháng       2024-04-06 00:00:00 27/6 2024 đã ký 2023,2024 đã ký HB Đã ký\n2 OEMHUNGT\nTrường: Hưng Thịnh\nMalayone   500000000      Đặt 30%\nTT ngay 2025-04-03 00:00:00          -\n3 OEMNGHIN\nA Nghinh\nSunshise 0 70000000 500000000   430000000   Đặt 30%\nTT ngay 2025-04-03 00:00:00      4/5\n4/6 13/6 2024 đã ký Không Không\n4 OEMHAVITECH\nHavitech 0  500000000   500000000   Đặt 30%\nTT ngay 2025-04-03 00:00:00          -\n5 OEMKAISE\nNgọc: Smarttech\nKaiser 100000000  500000000   600000000   Đặt 30%\nTT ngay 2

In [1]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core import VectorStoreIndex
from llama_index.readers.file import PandasExcelReader
from llama_index.core.node_parser import MarkdownElementNodeParser, SentenceSplitter
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# (Tuỳ chọn, nếu bạn vẫn muốn lưu vào Qdrant)
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

class DataParser:
    """
    Thay thế cho LlamaParse: Lớp này dùng PandasExcelReader từ llama_index
    để chuyển các file Excel thành list Document.
    """
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.reader = PandasExcelReader()

    def parse(self):
        documents = []
        for path in self.file_paths:
            docs = self.reader.load_data(file=path)
            documents.extend(docs)
        return documents


class NodeProcessor:
    """
    Tách/lọc các Document thành node. Sử dụng SentenceSplitter để chunk dữ liệu,
    kết hợp với MarkdownElementNodeParser nếu cần xử lý định dạng Markdown.
    """
    def __init__(self, 
                 llm_model='gemma3:4b', 
                 num_workers=10, 
                 chunk_size=5000, 
                 chunk_overlap=100, 
                 use_markdown_parser=False):
        """
        Khởi tạo NodeProcessor với tùy chọn chunk và parser.

        Args:
            llm_model (str): Tên mô hình LLM từ Ollama.
            num_workers (int): Số lượng worker để xử lý song song.
            chunk_size (int): Kích thước mỗi đoạn (số ký tự hoặc token).
            chunk_overlap (int): Độ chồng lấn giữa các đoạn.
            use_markdown_parser (bool): Sử dụng MarkdownElementNodeParser nếu True.
        """
        self.llm = Ollama(model=llm_model)
        
        # Khởi tạo parser chính để chunk dữ liệu
        self.chunk_parser = SentenceSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        
        # Tùy chọn sử dụng MarkdownElementNodeParser
        self.use_markdown_parser = use_markdown_parser
        if self.use_markdown_parser:
            self.markdown_parser = MarkdownElementNodeParser(
                llm=self.llm, 
                num_workers=num_workers
            )
        else:
            self.markdown_parser = None

    def process_nodes(self, documents):
        """
        Xử lý các Document thành node với cơ chế chunk.

        Args:
            documents (list): Danh sách các Document từ DataParser.

        Returns:
            base_nodes (list): Danh sách các node đã được tách.
            objects (list): Danh sách các object (nếu dùng Markdown parser).
        """
        # Bước 1: Chunk dữ liệu bằng SentenceSplitter
        chunked_nodes = self.chunk_parser.get_nodes_from_documents(documents)
        
        # Bước 2: Nếu dùng Markdown parser, xử lý thêm
        if self.use_markdown_parser and self.markdown_parser:
            base_nodes, objects = self.markdown_parser.get_nodes_and_objects(chunked_nodes)
            return base_nodes, objects
        else:
            # Chỉ trả về các node đã chunk nếu không dùng Markdown parser
            return chunked_nodes, []

class IndexBuilder:
    """
    Xây dựng Index (mặc định) hoặc có thể tích hợp với QdrantVectorStore.
    """
    def __init__(self, 
                 llm_model='gemma3:4b', 
                 embed_model_name='nomic-embed-text',
                 use_qdrant=False,
                 qdrant_host='localhost',
                 qdrant_port=6333,
                 collection_name='my_collection'):

        self.llm = Ollama(model=llm_model)
        self.embed_model = OllamaEmbedding(model_name=embed_model_name)

        # Tuỳ chọn sử dụng Qdrant
        self.use_qdrant = use_qdrant
        if self.use_qdrant:
            self.qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
            self.vector_store = QdrantVectorStore(
                client=self.qdrant_client,
                collection_name=collection_name,
                embeddings=self.embed_model
            )
        else:
            self.vector_store = None

    def build_index(self, nodes):
        if self.use_qdrant and self.vector_store is not None:
            # Index dùng Qdrant
            index = VectorStoreIndex(
                nodes=nodes,
                llm=self.llm,
                embed_model=self.embed_model,
                vector_store=self.vector_store
            )
        else:
            # Index cục bộ mặc định
            index = VectorStoreIndex(
                nodes=nodes, 
                llm=self.llm, 
                embed_model=self.embed_model
            )
        return index


from llama_index.core.prompts import PromptTemplate

class QueryEngine:
    """
    Tạo engine để truy vấn Index với prompt template tùy chỉnh.
    """
    def __init__(self, index, similarity_top_k=5, llm=None):
        # Định nghĩa prompt template
        custom_prompt_template = PromptTemplate(
            """
            Bạn là một trợ lý phân tích dữ liệu dự án xây dựng. Dựa trên thông tin từ các tài liệu sau: 
            {context_str}
            Hãy trả lời câu hỏi sau một cách chính xác, ngắn gọn và có căn cứ:
            {query_str}
            Nếu không đủ dữ liệu để trả lời, hãy nói rõ ràng rằng thông tin không có trong tài liệu.
            """
        )

        # Tạo query engine với prompt template
        self.query_engine = index.as_query_engine(
            similarity_top_k=similarity_top_k,
            llm=llm,
            text_qa_template=custom_prompt_template  # Thêm prompt template vào đây
        )

    def query(self, question):
        return self.query_engine.query(question)



# Danh sách các file Excel cần xử lý
excel_files = [
    # '/home/namnh1/rag-llm-chatbot/BC AI/BC Nhân sự.xlsx',
    # '/home/namnh1/rag-llm-chatbot/BC AI/BC Tiến độ sản xuất.xlsx',
    # '/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx',
    "/home/namnh1/rag-llm-chatbot/BC AI/BC Dự án.xlsx"
]

# 1. Tải dữ liệu Excel thành list Document
parser = DataParser(file_paths=excel_files)
documents = parser.parse()

# 2. Tách thành các node
processor = NodeProcessor()
base_nodes, objects = processor.process_nodes(documents)

# 3. Xây dựng Index (có thể bật tắt use_qdrant)
index_builder = IndexBuilder(
    llm_model='gemma3:12b', 
    embed_model_name='nomic-embed-text',
    use_qdrant=True,  # đặt True nếu muốn lưu vào Qdrant
    qdrant_host='localhost',
    qdrant_port=6333,
    collection_name='my_collection_test'
)
index = index_builder.build_index(base_nodes + objects)

# 4. Truy vấn
query_engine = QueryEngine(
    index=index, 
    similarity_top_k=5, 
    llm=index_builder.llm
)
response = query_engine.query("Dự án Sunshine Tháp C hiện tại công trường thi công đến giai đoạn nào? Bộ phận hồ sơ đã hoàn thành những thủ tục gì và còn những thủ tục nào cần tiếp tục xử lý?")

# 5. In kết quả
print(f"{response}")


Dưới đây là thông tin về tình trạng dự án Sunshine Tháp C dựa trên tài liệu đã cung cấp:

**1. Công trường thi công:**

*   Tài liệu không mô tả cụ thể giai đoạn thi công công trường.

**2. Tình trạng bộ phận hồ sơ:**

*   **Đã hoàn thành:**
    *   Hoàn thành hồ sơ thanh toán lắp đặt đợt 2 (IPC3) (Ngày 12/11, GT ĐNTT: 596.131.411 VNĐ).
    *   Hồ sơ quyết toán đã nhận bản scan, giá trị ĐNTT 95% thu về (742.978.195 VNĐ), giá trị bảo hành (275.763.439 VNĐ), và đã xuất hóa đơn ngày 19/02/2025.
*   **Còn phải tiếp tục/Đang thực hiện:**
    *   BBNT (Chứng nhận khối lượng hoàn thành) và BB thanh lý (Biến bản thanh lý hợp đồng) "đang chờ lãnh đạo ký" để phát hành thư bảo lãnh.
    *   Đang chốt công văn giải trình tiến độ (đã gửi để công ty ký đóng dấu).
    *   Thứ 2 chị Hà sẽ gửi cho TÙNG để trình ký phê duyệt Ban QLDA và triển khai các thủ tục quyết toán theo hợp đồng.


In [2]:
question = "So sánh tình hình công nợ thực tế và giá trị hợp đồng của hai dự án Sunshine Tháp C và Doji Hải Phòng. Dự án nào có tỷ lệ công nợ so với giá trị hợp đồng cao hơn? Hãy nhận xét rủi ro tài chính từ công nợ của từng dự án!"
response = query_engine.query(question)
response

Response(response='Dưới đây là so sánh và nhận xét dựa trên thông tin từ báo cáo:\n\n**1. So sánh công nợ thực tế và giá trị hợp đồng:**\n\n*   **Dự án Sunshine Tháp C:**\n    *   Giá trị HĐ/PLHĐ: 10,000,000,000 VNĐ\n    *   Công nợ thực tế: 3,845,243,040 VNĐ\n    *   Tỷ lệ công nợ: (3,845,243,040 / 10,000,000,000) * 100% = **38.45%**\n*   **Dự án Doji Hải Phòng:**\n    *   Giá trị HĐ: 20,000,000,000 VNĐ\n    *   Công nợ thực tế: 14,000,000,000 VNĐ\n    *   Tỷ lệ công nợ: (14,000,000,000 / 20,000,000,000) * 100% = **70%**\n\n**2. Nhận xét rủi ro tài chính:**\n\n*   **Sunshine Tháp C:** Mức công nợ 38.45% so với giá trị hợp đồng cho thấy dự án này đang được quản lý khá tốt và tiến độ thanh toán khá ổn định. Rủi ro tài chính được đánh giá ở mức trung bình.\n*   **Doji Hải Phòng:** Dự án có tỷ lệ công nợ 70%, cao hơn đáng kể so với giá trị hợp đồng.  Điều này đặt ra rủi ro tài chính cao hơn đối với dự án, có thể do tiến độ thanh quyết toán chậm trễ, phát sinh nhiều chi phí sửa chữa,... cầ

In [3]:
print(f"{response}")


Dưới đây là so sánh và nhận xét dựa trên thông tin từ báo cáo:

**1. So sánh công nợ thực tế và giá trị hợp đồng:**

*   **Dự án Sunshine Tháp C:**
    *   Giá trị HĐ/PLHĐ: 10,000,000,000 VNĐ
    *   Công nợ thực tế: 3,845,243,040 VNĐ
    *   Tỷ lệ công nợ: (3,845,243,040 / 10,000,000,000) * 100% = **38.45%**
*   **Dự án Doji Hải Phòng:**
    *   Giá trị HĐ: 20,000,000,000 VNĐ
    *   Công nợ thực tế: 14,000,000,000 VNĐ
    *   Tỷ lệ công nợ: (14,000,000,000 / 20,000,000,000) * 100% = **70%**

**2. Nhận xét rủi ro tài chính:**

*   **Sunshine Tháp C:** Mức công nợ 38.45% so với giá trị hợp đồng cho thấy dự án này đang được quản lý khá tốt và tiến độ thanh toán khá ổn định. Rủi ro tài chính được đánh giá ở mức trung bình.
*   **Doji Hải Phòng:** Dự án có tỷ lệ công nợ 70%, cao hơn đáng kể so với giá trị hợp đồng.  Điều này đặt ra rủi ro tài chính cao hơn đối với dự án, có thể do tiến độ thanh quyết toán chậm trễ, phát sinh nhiều chi phí sửa chữa,... cần có biện pháp giải quyết kịp thời 

In [4]:
question = "Đánh giá hiệu suất sử dụng nhân sự tại hai dự án: Với số lượng nhân sự khai thác, hồ sơ, giám sát và thợ thi công hiện có, liệu phân bổ nhân lực có hợp lý không? Có cần điều chỉnh nhân sự giữa các dự án để tối ưu tiến độ hay không"
response = query_engine.query(question)
print(f"{response}")

Dựa trên báo cáo dự án, đánh giá hiệu suất sử dụng nhân sự hai dự án như sau:

**KHÔNG ĐẦU. DỤC ĐÁNH GIÁ ĐƯỢC**

Tài liệu **không cung cấp** đủ thông tin để đánh giá tính hợp lý của việc phân bổ nhân sự giữa hai dự án và cần thiết phải điều chỉnh nhân sự để tối ưu tiến độ.

*   **Dự án Sunshine Tháp C:** Tài liệu liệt kê số lượng nhân sự: Nguyệt Bình, Hải, Bình (02 người), Tùng, Đức. Tuy nhiên không cung cấp số giờ làm việc hoặc sản lượng cụ thể của từng người, cũng như so sánh với tiến độ dự án.
*   **Dự án Doji Hải Phòng:** Liệt kê nhân sự: Hảo Liễu, Bình, Tuấn. Như trên, thiếu dữ liệu về hiệu suất và so sánh với tiến độ dự án.

Để có đánh giá chính xác, cần bổ sung thông tin về:

*   Số giờ làm việc/sản lượng thực tế của từng nhân sự
*   Mức độ hoàn thành các công việc so với kế hoạch
*   So sánh giữa hiệu suất nhân sự giữa hai dự án


## 1. Data Ingestion with DataParser
- Purpose:
Loads data from one or more Excel files.

- Key Component:

    - PandasExcelReader: Reads the Excel file(s) and converts each sheet or relevant section into a list of documents.
    - Workflow:
The parse() method iterates over the list of file paths, loads each file’s data, and aggregates the resulting documents into a single list.

## 2. Data Preprocessing with NodeProcessor
- Purpose:
Splits or “chunks” the raw documents into smaller, manageable text pieces (nodes) for efficient indexing and retrieval.

- Key Components:

    - SentenceSplitter: Splits the document text into chunks based on a defined size and overlap.
    - Optional Markdown Parsing:
If use_markdown_parser is set to True, an additional markdown parser (e.g., MarkdownElementNodeParser) is used to further process the nodes, preserving any markdown formatting.
    - Workflow:

    - Chunking: The process_nodes method first uses SentenceSplitter to break the documents into nodes.
Optional Markdown Processing: If enabled, it further processes these chunks to extract both nodes and supplementary objects.

In [15]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core import VectorStoreIndex
from llama_index.readers.file import PandasExcelReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine

class DataParser:
    def __init__(self, file_paths, sheet_names=None):
        self.file_paths = file_paths
        self.sheet_names = sheet_names
        self.reader = PandasExcelReader()

    def parse(self):
        documents = []
        for path in self.file_paths:
            try:
                excel_file = pd.ExcelFile(path)
                sheets = self.sheet_names if self.sheet_names else excel_file.sheet_names
                for sheet_name in sheets:
                    reader = PandasExcelReader(sheet_name=sheet_name)
                    docs = reader.load_data(file=path)
                    for doc in docs:
                        doc.metadata["sheet_name"] = str(sheet_name)
                        doc.metadata["file_path"] = path
                        documents.append(doc)
            except Exception as e:
                print(f"Error reading file {path}: {e}")
        return documents

    def get_sheet_names(self, file_path):
        try:
            excel_file = pd.ExcelFile(file_path)
            return excel_file.sheet_names
        except Exception as e:
            print(f"Error getting sheet names for {file_path}: {e}")
            return []

class NodeProcessor:
    """
    Tách/lọc các Document thành node. Sử dụng SentenceSplitter để chunk dữ liệu,
    kết hợp với MarkdownElementNodeParser nếu cần xử lý định dạng Markdown.
    """
    def __init__(self, 
                 llm_model='gemma3:4b', 
                 num_workers=10, 
                 chunk_size=5000, 
                 chunk_overlap=100, 
                 use_markdown_parser=False):
        """
        Khởi tạo NodeProcessor với tùy chọn chunk và parser.

        Args:
            llm_model (str): Tên mô hình LLM từ Ollama.
            num_workers (int): Số lượng worker để xử lý song song.
            chunk_size (int): Kích thước mỗi đoạn (số ký tự hoặc token).
            chunk_overlap (int): Độ chồng lấn giữa các đoạn.
            use_markdown_parser (bool): Sử dụng MarkdownElementNodeParser nếu True.
        """
        self.llm = Ollama(model=llm_model)
        
        # Khởi tạo parser chính để chunk dữ liệu
        self.chunk_parser = SentenceSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        
        # Tùy chọn sử dụng MarkdownElementNodeParser
        self.use_markdown_parser = use_markdown_parser
        if self.use_markdown_parser:
            self.markdown_parser = MarkdownElementNodeParser(
                llm=self.llm, 
                num_workers=num_workers
            )
        else:
            self.markdown_parser = None

    def process_nodes(self, documents):
        """
        Xử lý các Document thành node với cơ chế chunk.

        Args:
            documents (list): Danh sách các Document từ DataParser.

        Returns:
            base_nodes (list): Danh sách các node đã được tách.
            objects (list): Danh sách các object (nếu dùng Markdown parser).
        """
        # Bước 1: Chunk dữ liệu bằng SentenceSplitter
        chunked_nodes = self.chunk_parser.get_nodes_from_documents(documents)
        
        # Bước 2: Nếu dùng Markdown parser, xử lý thêm
        if self.use_markdown_parser and self.markdown_parser:
            base_nodes, objects = self.markdown_parser.get_nodes_and_objects(chunked_nodes)
            return base_nodes, objects
        else:
            # Chỉ trả về các node đã chunk nếu không dùng Markdown parser
            return chunked_nodes, []

class IndexBuilder:
    """
    Xây dựng Index, tích hợp với Qdrant nếu bật use_qdrant.
    """
    def __init__(self, 
                 llm_model='gemma3:4b', 
                 embed_model_name='nomic-embed-text',
                 use_qdrant=False,
                 qdrant_host='localhost',
                 qdrant_port=6333,
                 collection_name='my_collection'):
        self.llm = Ollama(model=llm_model)
        self.embed_model = OllamaEmbedding(model_name=embed_model_name)
        self.use_qdrant = use_qdrant
        self.collection_name = collection_name

        if self.use_qdrant:
            self.qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
            self._setup_qdrant_collection()
            self.vector_store = QdrantVectorStore(
                client=self.qdrant_client,
                collection_name=collection_name,
                embed_model=self.embed_model
            )
        else:
            self.vector_store = None

    def _setup_qdrant_collection(self):
        sample_text = "Test embedding size"
        sample_embedding = self.embed_model.get_text_embedding(sample_text)
        vector_size = len(sample_embedding)

        collections = self.qdrant_client.get_collections()
        if self.collection_name not in [col.name for col in collections.collections]:
            print(f"Creating collection '{self.collection_name}' in Qdrant...")
            self.qdrant_client.create_collection(
                collection_name=self.collection_name,
                vectors_config=rest.VectorParams(
                    size=vector_size,
                    distance=rest.Distance.COSINE
                )
            )
        else:
            print(f"Collection '{self.collection_name}' already exists.")

    def build_index(self, nodes):
        if self.use_qdrant and self.vector_store is not None:
            index = VectorStoreIndex(
                nodes=nodes,
                llm=self.llm,
                embed_model=self.embed_model,
                vector_store=self.vector_store
            )
            collection_info = self.qdrant_client.get_collection(self.collection_name)
            print(f"Pushed {collection_info.points_count} vectors to Qdrant collection '{self.collection_name}'.")
        else:
            index = VectorStoreIndex(
                nodes=nodes, 
                llm=self.llm, 
                embed_model=self.embed_model
            )
        return index

    def get_retriever(self, index, similarity_top_k=5):
        if self.use_qdrant and self.vector_store is not None:
            retriever = VectorIndexRetriever(
                index=index,
                vector_store=self.vector_store,
                similarity_top_k=similarity_top_k
            )
        else:
            retriever = VectorIndexRetriever(
                index=index,
                similarity_top_k=similarity_top_k
            )
        return retriever

from llama_index.core.response_synthesizers import get_response_synthesizer

class QueryEngine:
    """
    Tạo engine để truy vấn với retriever từ Qdrant hoặc index cục bộ.
    """
    def __init__(self, retriever, llm=None):
        custom_prompt_template = PromptTemplate(
            """
            Bạn là một trợ lý phân tích dữ liệu dự án xây dựng. Dựa trên thông tin từ các tài liệu sau: 
            {context_str}
            Hãy trả lời câu hỏi sau một cách chính xác, ngắn gọn và có căn cứ:
            {query_str}
            Nếu không đủ dữ liệu để trả lời, hãy nói rõ ràng rằng thông tin không có trong tài liệu.
            """
        )
        
        # Tạo response synthesizer với LLM (nếu có)
        response_synthesizer = get_response_synthesizer(
            llm=llm,
            text_qa_template=custom_prompt_template
        ) if llm else None

        # Khởi tạo RetrieverQueryEngine
        self.query_engine = RetrieverQueryEngine(
            retriever=retriever,
            response_synthesizer=response_synthesizer
        )
        self.retriever = retriever

    def query(self, question):
        retrieved_nodes = self.retriever.retrieve(question)
        print(f"Retrieved {len(retrieved_nodes)} nodes:")
        for i, node in enumerate(retrieved_nodes):
            print(f"Node {i+1}: {node.text[:100]}... (score: {node.score})")
        response = self.query_engine.query(question)
        return response


excel_files = ["/home/namnh1/rag-llm-chatbot/BC AI/BÁO_CÁO_DOANH_THU_GV_LN_2024_20_KH_OEM_CÓ_DT_LỚN_chatgpt.xlsm"]

# 1. Tải dữ liệu
parser = DataParser(file_paths=excel_files)
documents = parser.parse()

# 2. Tách thành nodes
processor = NodeProcessor(
    llm_model='gemma3:12b',
    num_workers=10,
    chunk_size=7000,
    chunk_overlap=1000
)
base_nodes, objects = processor.process_nodes(documents)

# 3. Xây dựng index
index_builder = IndexBuilder(
    llm_model='gemma3:12b', 
    embed_model_name='nomic-embed-text',
    use_qdrant=True,  # đặt True nếu muốn lưu vào Qdrant
    qdrant_host='localhost',
    qdrant_port=6333,
    collection_name='my_collection_test_2'
)
index = index_builder.build_index(base_nodes + objects)
retriever = index_builder.get_retriever(index, similarity_top_k=2)

# 4. Tạo query engine
query_engine = QueryEngine(
    retriever=retriever,
    llm=index_builder.llm
)

# 5. Truy vấn
question = "công ty 1 tổng số lượng thành phần A có số lượng là bao nhiêu ? Có doanh thu là bao nhiêu ? lợi nhuận bao nhiêu ?"
# question = "So sánh tình hình công nợ thực tế và giá trị hợp đồng của hai dự án Sunshine Tháp C và Doji Hải Phòng. Dự án nào có tỷ lệ công nợ so với giá trị hợp đồng cao hơn? Hãy nhận xét rủi ro tài chính từ công nợ của từng dự án!"
response = query_engine.query(question)
print(f"Response: {response}")

Collection 'my_collection_test_2' already exists.
Pushed 0 vectors to Qdrant collection 'my_collection_test_2'.
Retrieved 2 nodes:
Node 1: TỔNG HỢP                 
CÔNG TY CỔ PHẦN WILSON HÒA BÌNH                  
Thôn Sỏi, Xã Phú Thành, ... (score: 0.7393413654911803)
Node 2: 0 157.0 
99 156 Công ty 156 0 10000000 1444320 1444320 0 10000000 1.0 158.0 
23 157 Công ty 157 0 0 ... (score: 0.7113357957497177)
Response: Dựa trên thông tin tài liệu, Công ty 1 (trong danh mục "Khách hàng OEM") có:

*   **Tổng SL (Thành phẩm phụ kiện + tấm ốp):** 10109.91
*   **Tổng doanh thu:** 1183593250
*   **Tổng lợi nhuận:** 69322402


In [16]:
# 5. Truy vấn
question = "Công ty 1 có Số dư đầu kỳ bao nhiêu?"
# question = "So sánh tình hình công nợ thực tế và giá trị hợp đồng của hai dự án Sunshine Tháp C và Doji Hải Phòng. Dự án nào có tỷ lệ công nợ so với giá trị hợp đồng cao hơn? Hãy nhận xét rủi ro tài chính từ công nợ của từng dự án!"
response = query_engine.query(question)
print(f"Response: {response}")

Retrieved 2 nodes:
Node 1: TỔNG HỢP                 
CÔNG TY CỔ PHẦN WILSON HÒA BÌNH                  
Thôn Sỏi, Xã Phú Thành, ... (score: 0.655570507163369)
Node 2: 0 157.0 
99 156 Công ty 156 0 10000000 1444320 1444320 0 10000000 1.0 158.0 
23 157 Công ty 157 0 0 ... (score: 0.6342024460131701)


Response: Thông tin về "Số dư đầu kỳ" của Công ty 1 không có trong các tài liệu được cung cấp.



In [5]:
class DataParser:
    def __init__(self, file_paths, sheet_names=None):
        self.file_paths = file_paths
        self.sheet_names = sheet_names
        self.reader = PandasExcelReader()

    def parse(self):
        documents = []
        for path in self.file_paths:
            try:
                excel_file = pd.ExcelFile(path)
                sheets = self.sheet_names if self.sheet_names else excel_file.sheet_names
                for sheet_name in sheets:
                    reader = PandasExcelReader(sheet_name=sheet_name)
                    docs = reader.load_data(file=path)
                    for doc in docs:
                        doc.metadata["sheet_name"] = str(sheet_name)
                        doc.metadata["file_path"] = path
                        documents.append(doc)
            except Exception as e:
                print(f"Error reading file {path}: {e}")
        return documents

    def get_sheet_names(self, file_path):
        try:
            excel_file = pd.ExcelFile(file_path)
            return excel_file.sheet_names
        except Exception as e:
            print(f"Error getting sheet names for {file_path}: {e}")
            return []

In [8]:
excel_files = ["/home/namnh1/rag-llm-chatbot/BC AI/BÁO_CÁO_DOANH_THU_GV_LN_2024_20_KH_OEM_CÓ_DT_LỚN_chatgpt.xlsm"]

# 1. Tải dữ liệu
parser = DataParser(file_paths=excel_files)
documents = parser.parse()

In [9]:
documents

[Document(id_='35d41a28-3450-4d1d-9a33-05018d104265', embedding=None, metadata={'sheet_name': 'Sheet20', 'file_path': '/home/namnh1/rag-llm-chatbot/BC AI/BÁO_CÁO_DOANH_THU_GV_LN_2024_20_KH_OEM_CÓ_DT_LỚN_chatgpt.xlsm'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='           \nCÔNG TY CỔ PHẦN WILSON HÒA BÌNH           \nThôn Sỏi, Xã Phú Thành, Huyện Lạc Thuỷ, Tỉnh Hòa Bình, Việt Nam           \n           \n           \nBẢNG CÂN ĐỐI PHÁT SINH CÔNG NỢ           \nTài khoản: 131 - Phải thu khách hàng           \nTừ ngày 01/01/2024 đến ngày 31/12/2024           \n           \nStt Mã khách Tên khách Số dư đầu kỳ  Số phát sinh  Số dư cuối kỳ    \n   Nợ Có Nợ Có Nợ Có   kh 2 mã\n81 1 Công ty 1 9282605902 0 66539664163 67425243093 8397026972 0 1.0 1.0 \n109 2 Công ty 2 0 1700000000 19362139425 19362139425 0 1700000000 1.0 2.0 \n117 3 Cô